In [1]:
import os

http://cas.sdss.org/dr7/en/tools/crossid/crossid.asp
上述网站可以根据ra，dec返回run，rerun，camcol，field信息，这些信息可以根据对应在http://das.sdss.org/raw/中对应下载到
但是上述网站一次只能上传100KB的文件，经测试，200KB是最大承受范围，下面用来切割坐标列表，7000一组

In [ ]:
fits_dir = "/data/renhaoye/decals_2022/in_decals/fits/"
fits_files = os.listdir(fits_dir)
fits_files.sort()
step = 7000
iter = int(len(fits_files) / 7000)
for batch in range(iter + 1):
    with open("/data/renhaoye/decals_2022/decals_poi_7k%d.txt" % batch, "w") as f:
        if batch == iter:
            for i in range(batch * step, len(fits_files)):
                poi = fits_files[i].split(".fits")[0]
                ra, dec = poi.split("_")
                f.write("A" + str(i + 1) + " " + str(ra) + " " + str(dec) + "\n")
        else:
            for i in range(batch * step, (batch + 1) * step):
                poi = fits_files[i].split(".fits")[0]
                ra, dec = poi.split("_")
                # f.write(str(ra) + "," + str(dec) + "," + "1.117866\n")
                f.write("A" + str(i + 1) + " " + str(ra) + " " + str(dec) + "\n")

切割图像

In [2]:
import numpy as np
import astropy.units as u
import astropy
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.io import fits
# from astroquery.gaia import Gaia
import matplotlib.pyplot as plt
# Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3
%matplotlib inline

In [ ]:
PATH = "/data/renhaoye/decals_2022/"
hdul = fits.open(PATH + "g.fit")

data = hdul[0].data
header = hdul[0].header
wcsobj = WCS(hdul[0].header)
hdul.close()

In [ ]:
ra, dec = wcsobj.all_world2pix([[122.541511, 20.486553]], 0)[0]

In [3]:
sdss = pd.read_csv("/data/renhaoye/decals_2022/sdss_in_decals.csv")

In [5]:
sdss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299657 entries, 0 to 299656
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   col0        299657 non-null  object 
 1   objID       299657 non-null  float64
 2   ra          299657 non-null  float64
 3   dec         299657 non-null  float64
 4   run         299657 non-null  int64  
 5   rerun       299657 non-null  int64  
 6   camcol      299657 non-null  int64  
 7   field       299657 non-null  int64  
 8   type        299657 non-null  object 
 9   modelMag_u  299657 non-null  float64
 10  modelMag_g  299657 non-null  float64
 11  modelMag_r  299657 non-null  float64
 12  modelMag_i  299657 non-null  float64
 13  modelMag_z  299657 non-null  float64
dtypes: float64(8), int64(4), object(2)
memory usage: 32.0+ MB


In [68]:
import warnings
warnings.filterwarnings("ignore")
class Cut:
    def __init__(self, info):
        assert type(info)==pd.core.series.Series, "please input pd.core.series.Series like: df.iloc[index]"
        self.ra = info.ra
        self.dec = info.dec
        self.run = info.run
        self.rerun = info.rerun
        self.camcol = info.camcol
        self.field = info.field
        self.cutout = np.zeros((3, 256, 256))
    def __call__(self, *args, **kwargs):
        new_field = str(self.field)
        if len(new_field) == 2:
            new_field = "0" + new_field
        raw_path = "/data/renhaoye/sdss_dr7_decals_overlap/raw/"
        bands = ["g", "r", "z"]
        files = []
        for band in bands:
            files.append(raw_path + "fpC-00%s-%s%d-0%s.fit" % (self.run, band, self.camcol, new_field))
        # files = ["/data/renhaoye/decals_2022/g.fit", "/data/renhaoye/decals_2022/r.fit", "/data/renhaoye/decals_2022/z.fit"]
        for i in range(len(files)):
            with fits.open(files[i]) as hdul:
                data = hdul[0].data
                # print(data.shape)
                wcs = WCS(hdul[0].header)
                # print(hdul.info())
                y, x = wcs.all_world2pix([[self.ra, self.dec]],0)[0]
                # y, x = wcs.all_world2pix([[122.541511, 20.486553]], 0)[0]
                # assert x == np.nan or y == np.nan, "source not found in fit"
                x, y = int(x), int(y)
                # print(x, y)
                left_upper_x, left_upper_y = x - 128, y - 128
                # print(left_upper_x, left_upper_y)
                if left_upper_x > 1233 or left_upper_x < 0 or (left_upper_y > 1790) or left_upper_y<0:
                    self.cutout[i] = np.zeros((256, 256))
                else:
                    self.cutout[i] = data[left_upper_x:left_upper_x + 256, left_upper_y:left_upper_y+256]
        # self.cutout = np.array((self.cutout[0], self.cutout[1], self.cutout[2]))

In [92]:
def save_fits(data: np.ndarray, filename: str):
    """
    将ndarray保存成fits文件
    :param data: 待保存数据
    :param filename: 保存文件名
    :return:
    """
    if len(data.shape) == 2:
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename, overwrite=True)
        hdulist.close()
    elif data.shape[-1] == 3:
        g, r, z = data[:, :, 0], data[:, :, 1], data[:, :, 2]
        data = np.array((g, r, z))
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename, overwrite=True)
        hdulist.close()
    elif data.shape[0] == 3:
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename, overwrite=True)
        hdulist.close()
    else:
        raise RuntimeError

In [93]:
from tqdm import tqdm
# x = sdss.iloc[1]
for i in tqdm(range(len(sdss))):
    a = Cut(sdss.iloc[i])
    a()
    # max 2048*1489
    if np.max(a.cutout[0,:,:]) != 0. and np.max(a.cutout[1,:,:]) != 0. and np.max(a.cutout[2,:,:]) != 0.:
        ra = str(a.ra).split(".")[0] + "." + str(a.ra).split(".")[1][0:6]
        dec = str(a.dec).split(".")[0] + "." + str(a.dec).split(".")[1][0:6]
        save_fits(a.cutout, "/data/renhaoye/test/%s_%s.fits" % (ra, dec))

  2%|█▍                                                                          | 5759/299657 [10:09<8:38:41,  9.44it/s]


KeyboardInterrupt: 

In [79]:
type(a.ra)

numpy.float64

In [ ]:
num = 0
for item in max:
    if item == 0:
        num += 1
num

In [174]:
def save_fits(data: np.ndarray, filename: str):
    """
    将ndarray保存成fits文件
    :param data: 待保存数据
    :param filename: 保存文件名
    :return:
    """
    if len(data.shape) == 2:
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename)
        hdulist.close()
    elif data.shape[-1] == 3:
        g, r, z = data[:, :, 0], data[:, :, 1], data[:, :, 2]
        data = np.array((g, r, z))
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename)
        hdulist.close()
    elif data.shape[0] == 3:
        hdu = fits.PrimaryHDU(data)
        hdulist = fits.HDUList([hdu])
        hdulist.writeto(filename)
        hdulist.close()
    else:
        raise RuntimeError

In [175]:
save_fits(a.cutout, "/data/renhaoye/1.fits")

In [7]:
test = []
with open("/data/renhaoye/sdss_dr7_decals_overlap/done_index.txt", "r") as r:
    for i in r:
        test.append(i.split("\n")[0])

In [8]:
len(test)

217970

In [9]:
test

['11238',
 '37460',
 '44952',
 '29968',
 '14984',
 '22476',
 '48698',
 '26222',
 '3746',
 '18730',
 '63682',
 '59936',
 '52444',
 '71174',
 '67428',
 '11239',
 '22477',
 '71175',
 '33716',
 '56191',
 '41207',
 '7493',
 '11240',
 '37461',
 '29969',
 '14985',
 '48699',
 '63683',
 '26223',
 '52445',
 '3747',
 '18731',
 '22479',
 '56192',
 '11241',
 '33718',
 '41208',
 '7494',
 '44954',
 '59938',
 '56193',
 '37462',
 '22480',
 '14986',
 '29970',
 '63684',
 '26224',
 '3748',
 '52446',
 '71180',
 '18732',
 '11242',
 '48700',
 '33719',
 '67430',
 '56194',
 '44955',
 '41209',
 '59939',
 '37463',
 '22481',
 '14987',
 '63685',
 '26225',
 '52447',
 '3749',
 '71181',
 '48701',
 '11243',
 '6',
 '18733',
 '33720',
 '7496',
 '67431',
 '29972',
 '56195',
 '44956',
 '41210',
 '37464',
 '59940',
 '22482',
 '14988',
 '26226',
 '52448',
 '11244',
 '48702',
 '3750',
 '7',
 '71182',
 '33721',
 '67432',
 '29973',
 '44957',
 '56196',
 '63687',
 '37465',
 '41211',
 '59941',
 '22483',
 '18735',
 '14989',
 '7498